In [1]:
# Dependencies
import requests
from sqlalchemy import create_engine
import pandas as pd
import json
from datetime import datetime as dt

In [2]:
# Save config information.
url = "https://api.covid19api.com/summary"


In [3]:
# Pretty print JSON for all launchpads
response = requests.get(url).json()
#print(json.dumps(response, indent=4, sort_keys=True))


countries = response["Countries"]
print(json.dumps(countries, indent=4, sort_keys=True))



[
    {
        "Country": "ALA Aland Islands",
        "CountryCode": "AX",
        "Date": "2020-04-18T17:31:42Z",
        "NewConfirmed": 0,
        "NewDeaths": 0,
        "NewRecovered": 0,
        "Slug": "ala-aland-islands",
        "TotalConfirmed": 0,
        "TotalDeaths": 0,
        "TotalRecovered": 0
    },
    {
        "Country": "Afghanistan",
        "CountryCode": "AF",
        "Date": "2020-04-18T17:31:42Z",
        "NewConfirmed": 66,
        "NewDeaths": 0,
        "NewRecovered": 45,
        "Slug": "afghanistan",
        "TotalConfirmed": 906,
        "TotalDeaths": 30,
        "TotalRecovered": 99
    },
    {
        "Country": "Albania",
        "CountryCode": "AL",
        "Date": "2020-04-18T17:31:42Z",
        "NewConfirmed": 21,
        "NewDeaths": 0,
        "NewRecovered": 6,
        "Slug": "albania",
        "TotalConfirmed": 539,
        "TotalDeaths": 26,
        "TotalRecovered": 283
    },
    {
        "Country": "Algeria",
        "CountryCode":

In [4]:
# convert Json file to DataFrame
df = pd.DataFrame.from_dict(countries, orient='columns')
df.head()

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,ALA Aland Islands,AX,ala-aland-islands,0,0,0,0,0,0,2020-04-18T17:31:42Z
1,Afghanistan,AF,afghanistan,66,906,0,30,45,99,2020-04-18T17:31:42Z
2,Albania,AL,albania,21,539,0,26,6,283,2020-04-18T17:31:42Z
3,Algeria,DZ,algeria,150,2418,16,364,63,846,2020-04-18T17:31:42Z
4,American Samoa,AS,american-samoa,0,0,0,0,0,0,2020-04-18T17:31:42Z


In [5]:
# clean up data
df = df.drop('Slug', axis=1)
df.head()

,Country,CountryCode,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,ALA Aland Islands,AX,0,0,0,0,0,0,2020-04-18T17:31:42Z
1,Afghanistan,AF,66,906,0,30,45,99,2020-04-18T17:31:42Z
2,Albania,AL,21,539,0,26,6,283,2020-04-18T17:31:42Z
3,Algeria,DZ,150,2418,16,364,63,846,2020-04-18T17:31:42Z
4,American Samoa,AS,0,0,0,0,0,0,2020-04-18T17:31:42Z


In [6]:
# rename columns
df = df.rename(columns={"Country":"country","CountryCode":"country_code",
                        "NewConfirmed":"new_cases_confirmed","TotalConfirmed":"total_cases_confirmed","NewDeaths":"new_deaths",
                        "TotalDeaths":"total_deaths","NewRecovered":"new_recovered","TotalRecovered":"total_recovered",
                       "Date":"date"})
df.head()

,country,country_code,new_cases_confirmed,total_cases_confirmed,new_deaths,total_deaths,new_recovered,total_recovered,date
0,ALA Aland Islands,AX,0,0,0,0,0,0,2020-04-18T17:31:42Z
1,Afghanistan,AF,66,906,0,30,45,99,2020-04-18T17:31:42Z
2,Albania,AL,21,539,0,26,6,283,2020-04-18T17:31:42Z
3,Algeria,DZ,150,2418,16,364,63,846,2020-04-18T17:31:42Z
4,American Samoa,AS,0,0,0,0,0,0,2020-04-18T17:31:42Z


In [7]:
# format date
df['date'] = pd.to_datetime(df['date']).dt.date
df

,country,country_code,new_cases_confirmed,total_cases_confirmed,new_deaths,total_deaths,new_recovered,total_recovered,date
0,ALA Aland Islands,AX,0,0,0,0,0,0,2020-04-18
1,Afghanistan,AF,66,906,0,30,45,99,2020-04-18
2,Albania,AL,21,539,0,26,6,283,2020-04-18
3,Algeria,DZ,150,2418,16,364,63,846,2020-04-18
4,American Samoa,AS,0,0,0,0,0,0,2020-04-18
...,...,...,...,...,...,...,...,...,...
242,Wallis and Futuna Islands,WF,0,0,0,0,0,0,2020-04-18
243,Western Sahara,EH,0,6,0,0,0,0,2020-04-18
244,Yemen,YE,0,1,0,0,0,0,2020-04-18
245,Zambia,ZM,4,52,0,2,0,30,2020-04-18


In [8]:
#export to csv

#df.to_csv (r'C:\Users\Terry\Desktop\GT Repositories\covid_project\covid19.csv', index = False, header=True)

## Connect to local database

In [11]:
rds_connection_string = "postgres:postgres@localhost:5433/covid19_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

## Check for tables

In [12]:
engine.table_names()

['covid19', 'disease', 'covid20']

## Use pandas to load csv converted DataFrame into database

In [13]:
df.to_sql(name='disease', con=engine, if_exists='append', index=False)